In [1]:
############################# ASSIGNMENT PART ONE - UP TO LINE 6 ############################### 
################################################################################################

import pandas as pd
import numpy as np
import requests 

In [2]:
 #Pull wikipedia table into dataframe
url = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
toronto_data = pd.read_html(url)

In [3]:
#Remove all "Not assignd"
toronto = toronto_data[0][toronto_data[0].Borough != "Not assigned"]
toronto.shape

(211, 3)

In [4]:
#Group postal codes together, rename columns 

toronto2 = toronto.groupby(by=['Postcode','Borough']).agg(lambda x:', '.join(x))
toronto2.reset_index(level=['Postcode','Borough'], inplace=True)
toronto2.rename(columns={'Postcode': 'Postal Code','Neighbourhood':'Neighborhood'}, inplace=True)
toronto2.head()

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [5]:
#Assign Neighborhood Value as Borough if "Not assigned"

ix_msk = toronto2.loc[toronto2['Neighborhood'] == "Not assigned"].index

for i in ix_msk:
    toronto2.iloc[i, 2] = toronto2.iloc[i, 1]

In [6]:
toronto2.head()
toronto2.shape

(103, 3)

In [7]:
############################### ASSIGNMENT PART 2 - LINES 6 THROUGH 9 ############################# 
###################################################################################################

#Load latitude and Longitude values from CSV

lat_long = pd.read_csv("https://cocl.us/Geospatial_data")
lat_long.set_index('Postal Code').head()

,Latitude,Longitude
Postal Code,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476


In [8]:
#Set the index of the previous dataframe to postal code to match the new dataframe, then merge
toronto2.set_index('Postal Code')
toronto_ll = pd.merge(toronto2, lat_long, on='Postal Code')


In [9]:
toronto_ll.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [10]:
######################## ASSIGNMENT PART 3 - LINES 10 THROUGH 30  ################################# 
###################################################################################################

#Install relevant map tools

!pip install geopy
!pip install folium
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim
import folium # map rendering library
print('Libraries imported.')

     |████████████████████████████████| 92kB 12.3MB/s eta 0:00:01
Libraries imported.


In [11]:
#explore the dataset

print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(toronto_ll['Borough'].unique()),
        toronto_ll.shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighborhoods.


In [12]:
#Obtain coordinate values for Toronto

address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [13]:
#Filter to only Boroughs containing 'Toronto'

#toronto_ll = toronto_ll[toronto_ll['Borough'].str.contains('Toronto')]

In [14]:
# create map of Toronto
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_ll['Latitude'], toronto_ll['Longitude'], toronto_ll['Borough'], toronto_ll['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [15]:
# Define foursquare credentials

CLIENT_ID = 'AKCX2GTFV10033CD3WV2BEJMD1OCM2DOP0JWBMPGTNW54G0B' # your Foursquare ID
CLIENT_SECRET = 'UGZADPTBYFFWVG5R31GBEQBFKZTUGC5UMVGSYPXTI2C0GEGK' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [16]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [17]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius
toronto_venues = getNearbyVenues(names=toronto_ll['Neighborhood'],
                                   latitudes=toronto_ll['Latitude'],
                                   longitudes=toronto_ll['Longitude']
                                  )


Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens, Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West, 

In [18]:
print(toronto_venues.shape)
toronto_venues.head()

(2281, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Scarborough Historical Society,43.788755,-79.162438,History Museum
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store


In [19]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]


In [20]:
#group by neighborhood taking mean/frequencey of each category

toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()

In [21]:
# sorting function

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [22]:
# top 10 for each neighborhood

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)



In [23]:
# set number of clusters
kclusters = 4

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:200] 

array([1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1,
       1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1,
       1, 0, 1, 1, 1, 1, 2, 1, 0, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 2, 1, 1, 1, 1, 1, 0], dtype=int32)

In [24]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_ll

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood',how = 'right')


In [25]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [26]:
# CLUSTER 1
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Scarborough,0,Playground,Park,Asian Restaurant,Women's Store,Electronics Store,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant
23,North York,0,Convenience Store,Bank,Park,Women's Store,Ethiopian Restaurant,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant
30,North York,0,Airport,Park,Electronics Store,Women's Store,Ethiopian Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Empanada Restaurant
40,East York,0,Metro Station,Convenience Store,Park,Coffee Shop,Ethiopian Restaurant,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant
44,Central Toronto,0,Swim School,Dim Sum Restaurant,Park,Bus Line,Women's Store,Electronics Store,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Empanada Restaurant
48,Central Toronto,0,Trail,Park,Playground,Summer Camp,Electronics Store,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant
50,Downtown Toronto,0,Park,Trail,Playground,Building,Empanada Restaurant,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant
64,Central Toronto,0,Trail,Park,Sushi Restaurant,Jewelry Store,Empanada Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
72,North York,0,Japanese Restaurant,Pub,Sushi Restaurant,Park,Asian Restaurant,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant
74,York,0,Park,Women's Store,Fast Food Restaurant,Market,Festival,Field,Farmers Market,Falafel Restaurant,Event Space,Dog Run


In [27]:
# CLUSTER 2
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Scarborough,1,History Museum,Bar,Ethiopian Restaurant,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Event Space,Doner Restaurant
2,Scarborough,1,Pizza Place,Rental Car Location,Breakfast Spot,Medical Center,Mexican Restaurant,Electronics Store,Intersection,Cuban Restaurant,Cupcake Shop,Construction & Landscaping
3,Scarborough,1,Coffee Shop,Indian Restaurant,Korean Restaurant,Women's Store,Empanada Restaurant,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant
4,Scarborough,1,Lounge,Fried Chicken Joint,Bakery,Bank,Athletics & Sports,Thai Restaurant,Caribbean Restaurant,Hakka Restaurant,Falafel Restaurant,Event Space
5,Scarborough,1,Health & Beauty Service,Playground,Convenience Store,Empanada Restaurant,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
6,Scarborough,1,Convenience Store,Discount Store,Bus Station,Coffee Shop,Department Store,Women's Store,Empanada Restaurant,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
7,Scarborough,1,Bakery,Bus Line,Soccer Field,Park,Intersection,Fast Food Restaurant,Metro Station,Bus Station,Empanada Restaurant,Dumpling Restaurant
8,Scarborough,1,American Restaurant,Motel,Dog Run,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Women's Store
9,Scarborough,1,Café,College Stadium,General Entertainment,Skating Rink,Women's Store,Electronics Store,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant
10,Scarborough,1,Indian Restaurant,Latin American Restaurant,Pet Store,Vietnamese Restaurant,Chinese Restaurant,Light Rail Station,Electronics Store,Donut Shop,Drugstore,Dumpling Restaurant


In [28]:
# CLUSTER 3
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
25,North York,2,Park,Food & Drink Shop,Women's Store,Empanada Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Event Space
98,York,2,Park,Women's Store,Ethiopian Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Event Space


In [29]:
# CLUSTER 4
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,3,Fast Food Restaurant,Women's Store,Ethiopian Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Event Space


In [30]:
# After testing multiple k values it was determined that gains after a cluster size of about 4 tend to just create single-neighborhood clusters.
